In [1]:
import pandas as pd

csv_path = "/mnt/06.Brats/Brain-Tumors-Segmentation/nn-Former_CHJ_practice/csv/test_metrics.csv"
result_df = pd.read_csv(csv_path)

In [2]:
# result_df.head()

In [3]:
import pandas as pd
import nibabel as nib
import numpy as np
from tqdm import tqdm

# 예시 DataFrame (result_df)
# result_df = pd.read_csv('path_to_csv')  # 실제 CSV 파일 로드

# Label 매핑
num2label = {0: 'bg', 1: 'et', 2: 'tc', 3: 'wt'}

# 각 row 처리
for idx, row in tqdm(result_df.iterrows()):
    # ID와 파일 경로
    id = row['id']
    label_file = f"/workspace/brats2023/test/{id}/{id}-seg.nii.gz"

    # NIfTI 파일 로드
    label_nii = nib.load(label_file)

    # 데이터 배열 추출
    label_data = label_nii.get_fdata()

    # 고유 레이블 값과 갯수 계산
    labels, cnt = np.unique(label_data, return_counts=True)

    # 각 레이블 처리
    for label, count in zip(labels, cnt):
        # 조건 확인
        if label != 0 and (count == 0 or row[f'{num2label[label]}_dice'] == 0):
            print(f"Idx: {idx}, Error: {id}, Missing or Zero Dice for {num2label[label]}, Count: {count}, Dice: {row[f'{num2label[label]}_dice']}")



0it [00:00, ?it/s]

6it [00:01,  5.00it/s]

Idx: 4, Error: BraTS-GLI-00017-001, Missing or Zero Dice for tc, Count: 66343, Dice: 0.0


195it [00:40,  4.78it/s]

Idx: 194, Error: BraTS-GLI-01484-000, Missing or Zero Dice for et, Count: 135025, Dice: 0.0


...

Idx: 199, Error: BraTS-GLI-01515-000, Missing or Zero Dice for et, Count: 93990, Dice: 0.0


201it [00:41,  4.78it/s]

Idx: 200, Error: BraTS-GLI-01517-000, Missing or Zero Dice for et, Count: 65330, Dice: 0.0


209it [00:42,  4.87it/s]

Idx: 208, Error: BraTS-GLI-01666-000, Missing or Zero Dice for et, Count: 60126, Dice: 0.0


In [4]:
import pandas as pd
import nibabel as nib
import numpy as np
from tqdm import tqdm

csv_path = "/mnt/06.Brats/Brain-Tumors-Segmentation/nn-Former_CHJ_practice/csv/test_metrics.csv"
result_df = pd.read_csv(csv_path)
# Label 매핑
num2label = {0: 'bg', 1: 'et', 2: 'tc', 3: 'wt'}

# 각 row 처리
voxel_counts = []  # 모든 voxel count 데이터를 저장할 리스트

for idx, row in tqdm(result_df.iterrows(), total=result_df.shape[0]):
    # ID와 파일 경로
    id = row['id']
    label_file = f"/workspace/brats2023/test/{id}/{id}-seg.nii.gz"

    try:
        # NIfTI 파일 로드
        label_nii = nib.load(label_file)

        # 데이터 배열 추출
        label_data = label_nii.get_fdata()

        # 고유 레이블 값과 갯수 계산
        labels, cnt = np.unique(label_data, return_counts=True)

        # 각 레이블 voxel 갯수를 저장
        voxel_count = {num2label.get(label, f'label_{label}'): count for label, count in zip(labels, cnt)}

        # 누락된 레이블은 0으로 설정
        for label_name in num2label.values():
            if label_name not in voxel_count:
                voxel_count[label_name] = 0

        voxel_counts.append(voxel_count)

    except Exception as e:
        print(f"Error processing file {label_file}: {e}")
        voxel_counts.append({label_name: 0 for label_name in num2label.values()})  # 에러 발생 시 0으로 초기화

# DataFrame에 voxel 갯수 추가
voxel_counts_df = pd.DataFrame(voxel_counts)
result_df = pd.concat([result_df, voxel_counts_df], axis=1)

# 결과 확인
print(result_df.head())


100%|██████████| 209/209 [00:44<00:00,  4.72it/s]

                    id   et_dice   tc_dice   wt_dice       et_hd       tc_hd  \
0  BraTS-GLI-00003-000  0.863494  0.943825  0.935477    2.000000    2.828427   
1  BraTS-GLI-00009-001  0.801327  0.779964  0.657823    1.732051    2.000000   
2  BraTS-GLI-00011-000  0.874294  0.866396  0.910113    2.236068    2.449490   
3  BraTS-GLI-00012-000  0.008930  0.846206  0.926860   31.982792   32.206364   
4  BraTS-GLI-00017-001  0.000000  0.000000  0.852744  347.000000  347.000000   

       wt_hd   et_sens   tc_sens   wt_sens   et_spec   tc_spec   wt_spec  \
0   2.449490  0.940294  0.966165  0.966604  0.997220  0.998363  0.995064   
1  26.925823  0.986388  0.989111  0.511823  0.999750  0.999712  0.999437   
2   4.000000  0.992730  0.992208  0.880257  0.998754  0.998662  0.998131   
3  46.712929  0.912088  0.981383  0.984837  0.991219  0.979235  0.986324   
4   6.782330  1.000000  1.000000  0.761168  1.000000  1.000000  0.999429   

        bg      et     tc     wt  
0  8828761   17214  57773  

In [5]:
# result_df.head()

In [6]:
import pandas as pd
import nibabel as nib
import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import directed_hausdorff

csv_path = "/mnt/06.Brats/Brain-Tumors-Segmentation/nn-Former_CHJ_practice/csv/test_metrics.csv"
result_df = pd.read_csv(csv_path)

prediction_base_path = "/mnt/06.Brats/Brain_Tumors_Segmentation/stored_prediction/nn_former"
label_base_path = "/workspace/brats2023/test"

# Dice Coefficient 계산 함수
def dice_coefficient(y_true, y_pred):
    intersection = np.sum((y_true > 0) & (y_pred > 0))
    sum_gt_pred = np.sum(y_true > 0) + np.sum(y_pred > 0)
    if sum_gt_pred == 0:
        return 0.0  # Avoid division by zero
    return (2.0 * intersection) / sum_gt_pred

# Hausdorff95 Distance 계산 함수
def hausdorff95(y_true, y_pred):
    coords_true = np.argwhere(y_true > 0)
    coords_pred = np.argwhere(y_pred > 0)
    if coords_true.size == 0 or coords_pred.size == 0:
        return np.inf
    forward_hd = directed_hausdorff(coords_true, coords_pred)[0]
    reverse_hd = directed_hausdorff(coords_pred, coords_true)[0]
    return max(forward_hd, reverse_hd)

# Sensitivity 계산 함수
def sensitivity(y_true, y_pred):
    true_positive = np.sum((y_true > 0) & (y_pred > 0))
    false_negative = np.sum((y_true > 0) & (y_pred == 0))
    if true_positive + false_negative == 0:
        return 0.0
    return true_positive / (true_positive + false_negative)

# Specificity 계산 함수
def specificity(y_true, y_pred):
    true_negative = np.sum((y_true == 0) & (y_pred == 0))
    false_positive = np.sum((y_true == 0) & (y_pred > 0))
    if true_negative + false_positive == 0:
        return 0.0
    return true_negative / (true_negative + false_positive)

# 결과 저장 리스트
metrics = []


In [7]:
# 각 ID에 대해 계산
for idx, row in tqdm(result_df.iterrows(), total=result_df.shape[0]):
    id = row['id']
    prediction_file = f"{prediction_base_path}/{id}.nii.gz"
    label_file = f"{label_base_path}/{id}/{id}-seg.nii.gz"

    try:
        # Prediction 및 Label 파일 로드
        pred_nii = nib.load(prediction_file)
        label_nii = nib.load(label_file)

        pred_data = pred_nii.get_fdata()
        label_data = label_nii.get_fdata()

        # Metric 계산
        dice = dice_coefficient(label_data, pred_data)
        hd95 = hausdorff95(label_data, pred_data)
        sens = sensitivity(label_data, pred_data)
        spec = specificity(label_data, pred_data)

        # 결과 저장
        metrics.append({
            "id": id,
            "dice": dice,
            "hausdorff95": hd95,
            "sensitivity": sens,
            "specificity": spec
        })

    except Exception as e:
        print(f"Error processing {id}: {e}")
        metrics.append({
            "id": id,
            "dice": 0,
            "hausdorff95": np.inf,
            "sensitivity": 0,
            "specificity": 0
        })

# DataFrame 생성
metrics_df = pd.DataFrame(metrics)

# Mean 값 계산
mean_metrics = metrics_df.mean(numeric_only=True)
print("Mean Metrics:")
print(mean_metrics)

# 결과 저장
metrics_df.to_csv("calculated_metrics.csv", index=False)

  1%|          | 2/209 [00:00<00:26,  7.67it/s]

Error processing BraTS-GLI-00003-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 
Error processing BraTS-GLI-00009-001: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 


  2%|▏         | 4/209 [00:00<00:26,  7.70it/s]

Error processing BraTS-GLI-00011-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 
Error processing BraTS-GLI-00012-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 


  3%|▎         | 6/209 [00:00<00:26,  7.74it/s]

...

 31%|███       | 64/209 [00:08<00:19,  7.49it/s]

Error processing BraTS-GLI-00504-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 
Error processing BraTS-GLI-00512-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 


 31%|███       | 65/209 [00:08<00:19,  7.38it/s]

Error processing BraTS-GLI-00516-000: operands could not be broadcast together with shapes (240,240,155) (3,128,128,128) 


KeyboardInterrupt: 